### Data In Decorators

Some of the most valuable decorator patterns rely on using variables inside the decorator function itself. <br><br>

<u>This is not the same</u> as using variables inside the wrapper function. <br><br>

Imagine you need to keep a running average of what some function returns.

In [1]:
def running_average(func):
    data = {"total" : 0, "count" : 0}
    def wrapper(*args, **kwargs):
        val = func(*args, **kwargs)
        data["total"] += val
        data["count"] += 1
        print("Average of {} so far: {:.01f}".format(
        func.__name__, data["total"] / data["count"]))
        return func(*args, **kwargs)
    return wrapper

Each time the function is called, the average of all calls so far is printed out.<br><br>

<u>Decorator functions are called ***ONCE***</u> for each function they are applied to.<br><br>

***Then, each time that function is called in the code, the wrapper function is what’s actually executed.***<br><br>

So imagine applying it to a function like this:

In [2]:
@running_average
def foo(x):
    return x+2

foo(1)

Average of foo so far: 3.0


3

In [3]:
foo(10)

Average of foo so far: 7.5


12

In [4]:
foo(1)

Average of foo so far: 6.0


3

As you see, this creates an internal dictionary, named data, used to keep track of foo's metrics. <br><br><br>


The placement of data is important. Pop quiz:

    1. What happens if you move the line defining data up one line, outside the running_average function? 
    -> Then "data" will be global variable, the scope will not be restricted to the inside of "decorated" function.

    2. What happens if you that line down, into the wrapper function?
    -> Every time it will regenerate the data, not being able to keeping track of metrics.

#### So when exactly is running_average executed? 

The decorator function itself is executed exactly once for every function it decorates. If you "decorate" N functions, running_average is executed N times, so we get N different data dictionaries, each tied to one of the resulting decorated functions.<br><br>

This has nothing to do with how many times a decorated function is executed. The decorated function is, basically, one of the created wrapper functions.


#### what if you want to peek into data? 
The way we’ve written running_average, you can’t.<br>
data persists because of the reference inside of wrapper, <br>
but there is no way you can access it directly in normal Python code.<br><br>

But when you ***do*** need to do this, there is a very easy solution: <br>
simply assign data as an attribute to wrapper. For example:

In [5]:
# collectstats is much like running_average, but lets
# you access the data dictionary directly, instead
# of printing it out.
def collectstats(func):
    data = {"total" : 0, "count" : 0}
    def wrapper(*args, **kwargs):
        val = func(*args, **kwargs)
        data["total"] += val
        data["count"] += 1
        return func(*args, **kwargs)
    wrapper.data = data
    return wrapper

See that line 
    
    wrapper.data = data

Yes, you can do that - <u>A function in Python is just an object</u>. And in Python, you can add new attribute to objects by just assining them. This conveniently annotataes the decorated function:

In [6]:
@collectstats
def foo(x):
    return x+2
foo.data

{'total': 0, 'count': 0}

In [7]:
foo(1)

3

In [8]:
foo.data

{'total': 3, 'count': 1}

In [9]:
foo(2)
foo.data

{'total': 7, 'count': 2}

### Nonlocal
Let’s switch to a another problem you might run into.<br>
Here is an decorator that counts how many times a function has been called.

In [10]:
count = 0
def countcalls(func):
    def wrapper(*args, **kwargs):
        global count
        count += 1
        print('# of calls: {}'.format(count))
        return func(*args, **kwargs)
    return wrapper

@countcalls
def foo(x): return x + 2
@countcalls
def bar(x): return 3 * x

This version of countcalls has a bug, Can you spot it?<br>
-> Yes, it stores count as a global variable, meaning every function that is decorated will use the same variable.


In [11]:
foo(1)

# of calls: 1


3

In [12]:
foo(2)

# of calls: 2


4

In [13]:
bar(3)

# of calls: 3


9

But the solution is tricker than it seems. Here's one attempt

In [18]:
def countcalls(func):
    count=0
    def wrapper(*args, **kwargs):
        count+=1
        print(f"# of calls: {count}")
        return func(*args,**kwargs)
    return wrapper

@countcalls
def foo(x): 
    return x + 2

foo(1)

UnboundLocalError: local variable 'count' referenced before assignment

We can’t use global, because it’s not global. But in Python 3, we can use the nonlocal keyword:

In [19]:
def countcalls(func):
    count = 0
    def wrapper(*args, **kwargs):
        nonlocal count
        count += 1
        print('# of calls: {}'.format(count))
        return func(*args, **kwargs)
    return wrapper

@countcalls
def foo(x):
    return x+2

foo(1)

# of calls: 1


3

In [20]:
foo(2)

# of calls: 2


4

#### Why didn't we get error when trying "running_average" example?
For reference:

    def running_average(func):
        data={"total":0,"count":0}
        def wrapper(*args,**kwargs):
            val=func(*args,**kwargs)
            data["total"] +=val
            data["count"] +=1
            print("Average of {} so far: {:.01f}".format(
                func.__name__, data["total"] / data["count"])))
            return func("*args,**kwargs)
        return wrapper
    



When we have a line like :

    count +=1 

that's actually modifying the value of the count variable itself - because it really means: 

    count = count + 1

And whenever you modify(instaed of just read) a variable that was created in a larger scope, Python requires you to declare that's what you actually want, with:
- global
- nonlocal
<br>
<br>

Here is the sneaky thing tho: when you write:

    data["count"] +=1 

<u>This is not actually modifying data!</u> Or rather, it's not modifying the variable named data, which points to a dictionary object.<br><br>

Instaed, the statement **data["count"] +=1** invoked a method on the data object. This does change the state of the dictionary, but it doesn't make data point to a different dictionary object.<br><br>

**count+=1** on the other hand, makes "count" variable point to a different integer, so we need to use nonlocal there. 
